# **PFM2 – Modelado y Aplicación Práctica.**

### Punto de partida
Este notebook continúa directamente el trabajo realizado en las Fases 1–6, utilizando como insumo el dataset validado `data/processed/subset_modelado.parquet`.  
Dicho dataset incluye:  
- La demanda original y ajustada.  
- La etiqueta `is_outlier` (procedente de DBSCAN).  
- Las nuevas columnas de trazabilidad anual (`tipo_outlier_year` y `decision_outlier_year`) generadas en la Fase 6.  

Este punto de partida garantiza que el modelado se apoya sobre datos consistentes, libres de anomalías espurias y con información de contexto suficiente para interpretar señales de negocio.

### Objetivo
Entrenar y evaluar modelos de predicción de demanda robustos, comparando diferentes enfoques (modelos estadísticos, machine learning y enfoques híbridos) y evaluando su capacidad para:  
- Integrar señales clave como top ventas y eventos de calendario.  
- Capturar tendencias, estacionalidades y picos de forma coherente.  
- Servir como base para la construcción de una aplicación interactiva en **Streamlit**, que permita al usuario explorar, simular y consumir las previsiones en un entorno operativo.  

### Referencia metodológica
Para una descripción detallada del tratamiento de outliers y validaciones aplicadas, ver `reports/outliers/outliers_resumen.csv` y el notebook de Fase 6.

---

### **Índice de Contenidos**

#### Fase 1: Inspección y preparación de datasets de partida
- 1.1. Reetiquetado temporal de la previsión de demanda (2025).
- 1.2. Revisión y reetiquetado del histórico 2023.
- 1.3. Revisión y reetiquetado del histórico 2022.
- 1.4. Limpieza y preparación del catálogo de productos.
- 1.5. Coherencia entre los datos históricos.
  - 1.5.1. Análisis visual.
  - 1.5.2. Análisis estadístico.
  - 1.5.3. Contraste de hipótesis.
  - 1.5.4. Conclusiones finales.

#### Fase 2: Desagregación de los datos
- 2.1. Generación del patrón estacional para la desagregación de la demanda.
  - 2.1.1. Aplicación del patrón estacional por año (2022–2024).
  - 2.1.2. Validación del calendario estacional.
- 2.2. Aplicación del patrón estacional a la demanda anual.
  - 2.2.1. Desagregación diaria del año 2024.
  - 2.2.2. Desagregación diaria del año 2023.
  - 2.2.3. Desagregación diaria del año 2022.
  - 2.2.4. Conclusiones de la desagregación de demanda diaria.
- 2.3. Comparativa entre años: ¿se ha aplicado bien la estacionalidad?
  - 2.3.1. Evolución diaria total por año (curva cruda + suavizada).
  - 2.3.2.  Correlación de las curvas diarias agregadas (2022–2024).
  - 2.3.3. Demanda media diaria mensual por año.
  - 2.3.4. KPIs de consistencia (CV mensual y correlaciones).
  - 2.3.5. Validación extra con calendario real (España 2022–2024).
  - 2.3.6. Conclusiones de la validación estacional y configuración definitiva.

#### Fase 3: Construcción del subset representativo.
- 3.1. Unificación de demandas (2022–2024).
- 3.2. Cruce con catálogo y asociación de categorías.
- 3.3. Filtrado de casos problemáticos.
- 3.4. Reducción de dimensionalidad (PCA sobre categorías).
- 3.5. Clustering de productos.
- 3.6. Generación del subset representativo.

#### Fase 4: Impacto del precio sobre la demanda.
- 4.1. Objetivo, datos de partida y mapeo de columnas y diseños del efecto precio (ventanas + elasticidades).
- 4.2. Preflight de ventanas — `ventanas_precio.py`
- 4.3. Aplicación del efecto — `aplicar_efecto_precio.py`
- 4.4. Validación rápida (sanity).
- 4.5. Validación adicional: alineamiento con calendario real.

#### Fase 5: Aplicación de factores externos y simulación de escenarios.
- 5.1. Introducción y objetivos.
- 5.2. Definición de factores externos.
- 5.3. Diseño del modelo de aplicación.
- 5.4. Implementación en código.
- 5.5. Validación de coherencia y robustez.
  - 5.5.1. Validación de coherencia del precio.
  - 5.5.2. Validación adicional (alineamiento ventanas).
  - 5.5.3. Comparativa de demanda.
  - 5.5.4. Validación de trazabilidad.
- 5.6. Conclusiones de la fase 5.

#### Fase 6: Análisis y tratamiento de outliers.
- 6.1. Validación complementaria: búsqueda de nuevos candidatos.
- 6.2. Análisis de outliers detectados por DBSCAN.
- 6.3. Resultados consolidados y decisiones finales.
- 6.4. Implicaciones para el modelado.
  - 6.4.1. Integración en el subset final.
  - 6.4.2. Visualización del impacto de outliers.


-----

⚠️ **Nota metodológica sobre los datos históricos utilizados.**

Los datos históricos correspondientes a los ejercicios 2022–2024 no proceden de registros reales de ventas, sino que fueron **generados a partir de la previsión de demanda 2025**. 
Para construir estos históricos se aplicaron de manera controlada diversos componentes que reflejan el comportamiento esperado en un contexto de comercio electrónico:

- **Patrón estacional**: incorporación de estacionalidad diaria y anual (ciclos de ingresos mensuales, rebajas, campañas como Black Friday, Prime Day, etc.).

- **Impacto del precio**: simulación del efecto del precio sobre la demanda, con distinta sensibilidad por clúster de producto.

- **Factores externos**: inclusión de variables de calendario y eventos promocionales como dummies exógenas.

- **Ruido controlado y aleatorio**: añadido de perturbaciones aleatorias con distribución normal, calibradas para introducir variabilidad sin distorsionar las tendencias de fondo.

> Este enfoque buscó **evitar la circularidad** inherente a la construcción de históricos a partir de una previsión futura, de manera que los modelos no aprendan relaciones deterministas y conserven capacidad de generalización.

🛑 **Limitaciones**

No obstante, este planteamiento presenta ciertas limitaciones que deben ser tenidas en cuenta en la interpretación de los resultados:

- Los datos de 2022–2024 heredan en gran medida las tendencias y estacionalidades de la previsión 2025, lo que puede reducir la 
  diversidad de patrones respecto a históricos reales.

- El ruido introducido, aunque aleatorio, no refleja en su totalidad la complejidad de desviaciones reales  
  (errores humanos, incidencias logísticas, cambios imprevistos de mercado).

- La validación mediante backtesting sobre 2024 se realiza frente a un histórico simulado a partir de 2025, lo que podría generar resultados 
  algo más optimistas que en un entorno con datos 100% reales.

🔍 **Enfoque adoptado**

A pesar de estas limitaciones, el enfoque es **válido y adecuado** para los objetivos del proyecto porque:

- Permite **evaluar de manera realista la metodología de predicción y el pipeline completo**(desde la generación de features hasta la selección de modelos).

- Introduce suficiente variabilidad y ruido para que los algoritmos deban **aprender patrones** y no simplemente replicar la previsión original.

- Facilita la comparación objetiva entre diferentes familias de modelos y la selección por clúster en base a métricas robustas (sMAPE, WAPE, MAE ponderado).

> En conclusión, los históricos generados proporcionan un marco de prueba **coherente y consistente** para validar la lógica del sistema de predicción y simulación de stock, 
entendiendo que los resultados no equivalen a un backtesting sobre datos 100% reales, sino a un escenario controlado que reproduce condiciones verosímiles.

📌 **Nota metodológica final sobre outliers y clusters**

En la Fase 2, a partir del clustering con DBSCAN, un conjunto reducido de productos quedó marcado como outliers. En lugar de eliminarlos del subset (como se hizo en clase), se decidió mantenerlos en el dataset, ya que el análisis posterior mostró que estos productos coincidían con dos situaciones:

- **Top ventas** → productos de alta rotación cuya exclusión hubiera distorsionado la demanda real.
- **Picos aislados coherentes** → ventas puntuales pero justificadas por campañas, estacionalidad o ventanas de grandes ventas.

Durante la Fase 6, para garantizar que todos los productos participaran en el modelado por clúster, se creó la columna __cluster__.

- En los productos no outliers (is_outlier = 0), cluster y __cluster__ son idénticos.
- En los productos outliers (is_outlier = 1), se aplicó un **criterio de fallback determinista**, asignándolos al clúster mayoritario (clúster 1).

**Limitaciones**

- Este enfoque diluye en cierta medida la especificidad de los outliers.
- Sin embargo, dado que en este caso **todos los outliers estaban justificados** (bien por ser top ventas, bien por picos coherentes con la época), su integración en el clúster mayoritario no compromete la validez del modelo.

**Enfoque adoptado**

- Se opta por mantener la asignación al clúster mayoritario para no dejar productos fuera del pipeline.
- Se documenta esta decisión como un compromiso entre simplicidad, cobertura y coherencia de negocio.
- Como línea futura, se podría explorar una reasignación basada en distancias a centroides u otras métricas, pero no se considera necesaria en esta fase.


**Posible línea futura: clustering específico de outliers**

En el presente proyecto los productos identificados como outliers fueron integrados en el clúster mayoritario con el objetivo de garantizar su cobertura en 
el modelado y evitar su eliminación, dado que en su mayoría correspondían a top ventas o a picos de demanda coherentes con la estacionalidad.

Como línea de trabajo futura, se podría plantear un clustering específico sobre el conjunto de outliers. Esta estrategia permitiría identificar subgrupos internos 
(por ejemplo, distinguir entre productos con alta rotación recurrente frente a productos con picos estacionales aislados) y, en consecuencia, aplicar modelos diferenciados más ajustados a cada comportamiento.

No obstante, dado que el volumen de productos outliers es reducido respecto al total (alrededor de un 5–6 %) y que los modelos con variables exógenas ya permiten explicar 
sus patrones de manera satisfactoria, se considera que esta extensión no es necesaria en la versión actual del modelo y se pospone como línea futura de refinamiento.

## FASE 7: **Validación y preparación del dataset para el modelado**.

### **7.1. Validación inicial del dataset.**


El primer paso antes de comenzar con el modelado consiste en realizar una **validación exhaustiva del dataset de partida**.  
El objetivo de este bloque es garantizar que los datos sobre los que se entrenarán los modelos son **consistentes, completos y utilizables**, evitando que errores estructurales condicionen los resultados posteriores.

🎯 **Objetivo**
- Comprobar que la **variable objetivo** (`demand_final_noised`) no presenta valores nulos ni negativos.
- Verificar que las **fechas** cubren el rango esperado (2022–2024) y que no existen duplicados en la combinación (`product_id`, `date`).
- Identificar posibles problemas de cobertura temporal (fechas faltantes, series constantes, productos incompletos).
- Validar que todos los **productos tienen un clúster asignado** y que la información de outliers está correctamente registrada.
- Revisar de forma preliminar las **variables de precio y factores externos**.

❓ **Por qué se realiza**
Una validación previa es esencial porque:
- Asegura que los **modelos trabajen con datos coherentes** y sin inconsistencias.
- Evita que los resultados del backtesting estén sesgados por errores de entrada.
- Permite identificar productos o periodos problemáticos antes de invertir tiempo en el entrenamiento.

🛠️ **Cómo se lleva a cabo**
La validación se efectúa mediante un **script específico** (`validacion_dataset_modelado.py`) que genera un reporte con:
- Información general del dataset.
- Estado de la variable objetivo.
- Cobertura temporal por producto.
- Comprobaciones sobre clústeres y outliers.
- Un **resumen tipo semáforo** (OK/NO-OK) de las validaciones críticas.

> De esta manera, cualquier problema estructural queda documentado y puede ser corregido antes de pasar a la fase de preparación de datos.


⚙️ **Script: `validacion_dataset_modelado.py`**

🎯 **Objetivo.**  
Automatizar la validación del dataset de modelado, comprobando la integridad de la variable objetivo, la cobertura temporal, los clústeres y la trazabilidad de los outliers. Este script actúa como herramienta de diagnóstico previa al modelado.

➡️ **Entradas.**
- `data/processed/subset_modelado.parquet` (dataset validado en Fases 1–6).

⬅️ **Salidas.**
- Reporte en consola con todos los resultados de validación.  
- (Opcional) Archivo TXT si se especifica `--report`.

🔁 **Flujo de trabajo.**
1. **Carga del dataset** (Parquet).  
2. **Chequeo de columnas y tipos** (`df.info()` capturado en buffer).  
3. **Validación de la variable objetivo**: nulos, negativos, estadísticos básicos.  
4. **Cobertura temporal**: fechas mínimas/máximas globales y por producto; detección de duplicados `product_id+date`; cálculo de completitud diaria.  
5. **Series constantes**: identifica productos con demanda sin variación.  
6. **Precio y factores**: detección de valores nulos/negativos en columnas relevantes (`precio_medio`, `price_factor_effective`).  
7. **Validación de clústeres**: confirmación de que todos los productos tienen clúster asignado; coherencia `cluster` vs `__cluster__` en productos no-outlier.  
8. **Outliers**: verificación de columnas relacionadas, recuento de productos marcados y número de clústeres asignados.  
9. **Resumen “semáforo”**: indicadores booleanos (`OK=True/False`) de las comprobaciones críticas.

🪛 **Parámetros modificables.**
- Rutas de entrada y salida (`--in`, `--report`).
- Nombre de la variable objetivo (`demand_final_noised` por defecto).

🧩 **Ejecución.**
- CLI:  
  ```bash
  python scripts/eda/validacion_dataset_modelado.py
  python scripts/eda/validacion_dataset_modelado.py --report reports/validacion_dataset.txt

- Notebook:

 `from scripts.eda.validacion_dataset_modelado import run_validation`
 
 `print(run_validation())`

📝 **Notas.**
- El script no modifica el dataset original.
- Si se encuentra algún problema crítico (ej. nulos en target, fechas fuera de rango), debe ser corregido antes de continuar con el modelado.

In [6]:
# =============================================================================
# Script: validación_dataset_modelado.py
# Validación inicial del dataset de modelado
# Objetivo: foto rápida y completa de calidad de datos y trazabilidad de clúster/outliers
# =============================================================================


from __future__ import annotations
from pathlib import Path
import argparse
import logging
import io
import pandas as pd
import numpy as np

# ---------- Helper: encontrar raíz del repo (carpeta que contenga data/processed) ----------
def find_repo_root(start: Path | None = None) -> Path:
    p = Path(start or Path.cwd()).resolve()
    for parent in (p, *p.parents):
        if (parent / "data" / "processed").exists():
            return parent
    return p  # fallback: cwd si no encuentra nada

# ---------- Rutas por defecto (funciona en script y en notebook) ----------
if "__file__" in globals():
    _start = Path(__file__).resolve().parent
    LOGGER_NAME = Path(__file__).stem
else:
    _start = Path.cwd()
    LOGGER_NAME = "notebook.validacion_dataset_modelado"

ROOT_DIR = find_repo_root(_start)
PROCESSED_DIR = ROOT_DIR / "data" / "processed"

# ---------- Logging ----------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(name)s | %(message)s",
)
log = logging.getLogger(LOGGER_NAME)


# ---------- Núcleo de validación ----------
def validate_dataset(df: pd.DataFrame, target: str = "demand_final_noised") -> str:
    """Devuelve un string con el reporte de validación."""
    lines: list[str] = []

    # 1) Columnas / tipos
    lines.append("=== ENCABEZADOS ===")
    lines.append(str(list(df.columns)))

    lines.append("\n=== INFO ===")
    buf = io.StringIO()                       # <- buffer válido para df.info()
    df.info(buf=buf, show_counts=True)
    lines.extend(buf.getvalue().splitlines())

    # 2) Target
    assert target in df.columns, f"No existe la columna objetivo '{target}'"
    tgt = df[target]
    lines.append(f"\n=== TARGET ({target}) ===")
    lines.append(f"Nulos: {int(tgt.isna().sum())}")
    lines.append(f"Negativos: {int((tgt < 0).sum())}")
    lines.append(str(tgt.describe()))

    # 3) Fechas y cobertura
    assert "date" in df.columns, "Falta columna 'date'"
    df = df.copy()
    df["date"] = pd.to_datetime(df["date"])
    lines.append("\n=== COBERTURA GLOBAL DE FECHAS ===")
    lines.append(f"Min: {df['date'].min()}  |  Max: {df['date'].max()}")

    # Duplicados product_id+date
    dups = int(df.duplicated(["product_id", "date"]).sum())
    lines.append(f"\nDuplicados (product_id, date): {dups}")

    # Continuidad diaria por producto
    span = df.groupby("product_id")["date"].agg(["min", "max", "count"])
    span["dias_esperados"] = (span["max"] - span["min"]).dt.days + 1
    span["completitud_%"] = (span["count"] / span["dias_esperados"] * 100).round(2)
    faltantes = int((span["completitud_%"] < 100).sum())
    lines.append(f"Productos con fechas faltantes: {faltantes}")
    lines.append(f"Completitud media %: {span['completitud_%'].mean().round(2)}")

    # Series constantes
    var0 = int((df.groupby("product_id")[target].nunique() == 1).sum())
    lines.append(f"Productos con demanda constante (único valor): {var0}")

    # 4) Precio (si existe)
    for col in ["precio_medio", "price_factor_effective"]:
        if col in df.columns:
            lines.append(f"\n=== CHEQUEO {col} ===")
            lines.append(
                f"Nulos: {int(df[col].isna().sum())} | Negativos: {int((df[col] < 0).sum())} "
                f"| Min: {df[col].min()} | Max: {df[col].max()}"
            )

    # 5) Clústeres
    cluster_col = "__cluster__" if "__cluster__" in df.columns else ("cluster" if "cluster" in df.columns else None)
    assert cluster_col is not None, "No hay columna de cluster ni __cluster__"
    lines.append(f"\n=== CLÚSTERES ({cluster_col}) ===")
    lines.append(f"Productos únicos: {df['product_id'].nunique()}")
    lines.append(f"Productos con cluster: {df.loc[df[cluster_col].notna(), 'product_id'].nunique()}")
    lines.append(f"Productos SIN cluster: {df.loc[df[cluster_col].isna(), 'product_id'].nunique()}")

    # Coherencia en NO-outliers
    if {"cluster", "__cluster__", "is_outlier"}.issubset(df.columns):
        no_out = df["is_outlier"].eq(0)
        iguales = (df.loc[no_out, "cluster"].fillna(-1).astype(int)
                   == df.loc[no_out, "__cluster__"].astype(int)).all()
        lines.append(f"Cluster y __cluster__ idénticos en NO-outliers: {bool(iguales)}")

    # 6) Outliers
    outlier_cols = [c for c in df.columns if "outlier" in c.lower()]
    lines.append("\n=== COLUMNAS OUTLIERS ===")
    lines.append(str(outlier_cols))
    if "is_outlier" in df.columns:
        n_out = int(df.query("is_outlier == 1")["product_id"].nunique())
        lines.append(f"Productos outlier: {n_out}")
        asign = df.loc[df["is_outlier"] == 1, ["product_id", cluster_col]].drop_duplicates()
        lines.append(f"Clusters distintos en outliers: {asign[cluster_col].nunique()}")

    # 7) Resumen semáforo
    checks = {
        "target_sin_nulos": int(tgt.isna().sum()) == 0,
        "target_sin_negativos": int((tgt < 0).sum()) == 0,
        "sin_duplicados_pid_fecha": dups == 0,
        "cluster_cubierto": df.loc[df[cluster_col].isna(), "product_id"].nunique() == 0,
    }
    lines.append("\n=== RESUMEN (OK=True) ===")
    for k, v in checks.items():
        lines.append(f"{k}: {bool(v)}")

    return "\n".join(lines)


# ---------- CLI (ignora flags extra de Jupyter) ----------
def _parse_args() -> argparse.Namespace:
    p = argparse.ArgumentParser(description="Validación inicial del dataset de modelado (no escribe por defecto).")
    p.add_argument("--in", dest="inp", type=str, default=str(PROCESSED_DIR / "subset_modelado.parquet"),
                   help="Ruta de entrada (PARQUET).")
    p.add_argument("--report", dest="report", type=str, default="",
                   help="Ruta TXT para volcar el reporte (opcional).")
    args, _ = p.parse_known_args()
    return args


# ---------- Atajo para usar desde notebook ----------
def run_validation(inp: str | Path = None, report: str | Path = None) -> str:
    inp_path = Path(inp) if inp else (PROCESSED_DIR / "subset_modelado.parquet")
    log.info("Leyendo: %s", inp_path)
    df = pd.read_parquet(inp_path)
    log.info("Validando…")
    rep = validate_dataset(df)
    if report:
        report = Path(report)
        report.parent.mkdir(parents=True, exist_ok=True)
        report.write_text(rep, encoding="utf-8")
        log.info("Reporte guardado en: %s", report)
    return rep


def main() -> None:
    args = _parse_args()
    txt = run_validation(args.inp, args.report)
    print(txt)


if __name__ == "__main__":
    main()


2025-09-08 11:29:18,994 | INFO | notebook.validacion_dataset_modelado | Leyendo: C:\Users\crisr\Desktop\Máster Data Science & IA\PROYECTO\PFM2_Asistente_Compras_Inteligente\data\processed\subset_modelado.parquet
2025-09-08 11:29:19,657 | INFO | notebook.validacion_dataset_modelado | Validando…


=== ENCABEZADOS ===
['precio_medio', 'product_id', 'demand_day', 'is_outlier', 'cluster', 'date', '__cluster__', '__product_id__', 'demand_multiplier', 'demand_day_priceadj', 'price_factor_effective', 'price_virtual', 'm_agosto_nonprice', 'm_competition', 'm_inflation', 'm_promo', 'm_seasonextra', 'm_segments', 'demand_final', 'factors_applied', 'demand_final_noised', 'demand_final_noiseds_adj', 'year', 'tipo_outlier_year', 'decision_outlier_year']

=== INFO ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3941216 entries, 0 to 3941215
Data columns (total 25 columns):
 #   Column                    Non-Null Count    Dtype         
---  ------                    --------------    -----         
 0   precio_medio              3941216 non-null  float64       
 1   product_id                3941216 non-null  string        
 2   demand_day                3941216 non-null  float64       
 3   is_outlier                3941216 non-null  int64         
 4   cluster                   37209

📊 **Resultados de la validación inicial del dataset**

La validación aplicada sobre `subset_modelado.parquet` confirma que el dataset de partida es **consistente y apto para el modelado**.  

**Principales resultados:**
- ✔️ **Estructura completa**: se detectaron 25 columnas, incluyendo demanda, producto, clústeres, precios y factores externos.  
- ✔️ **Variable objetivo (`demand_final_noised`)**: sin valores nulos ni negativos.  
- ✔️ **Integridad temporal**: fechas cubren el rango esperado (2022–2024), sin duplicados en la combinación (`product_id`, `date`).  
- ✔️ **Cobertura de clústeres**: todos los productos tienen un clúster asignado.  
- ✔️ **Factores de precio y externos**: columnas presentes y sin anomalías graves.  

**Implicaciones para el modelado:**
- El dataset puede utilizarse directamente en la preparación (fase 7.2) sin necesidad de limpieza adicional.  
- La ausencia de nulos/duplicados evita sesgos en el backtesting y facilita la comparabilidad de métricas.  
- La cobertura de clústeres garantiza que se pueda aplicar el enfoque de modelado **por clúster**, manteniendo consistencia metodológica.  

> En conclusión, el dataset validado ofrece una **base sólida y coherente** para iniciar la fase de modelado, reduciendo riesgos de errores estructurales.

Además de la validación principal, se cuenta con un **script específico** (`check_outliers_clusters.py`)para auditar la coherencia de los *outliers* respecto a los clústeres.

**Objetivo.**  
Comprobar que:
- Los productos no marcados como *outliers* mantienen coherencia entre `cluster` y `__cluster__`.
- Los productos marcados como *outliers* tienen un clúster asignado y se registra correctamente su distribución.

**Entradas.**
- `data/processed/subset_modelado.parquet`

**Salidas.**
- Reporte en consola con:
  - Distribución de clústeres.
  - Coherencia `cluster` vs `__cluster__` en productos no-outlier.
  - Resumen de productos outlier y clústeres asignados.

**Uso.**
- CLI:
  ```bash
  python scripts/eda/check_outliers_clusters.py
  python scripts/eda/check_outliers_clusters.py --report reports/outliers/summary_outliers_clusters.txt

**Notas.**

- Este script se considera una herramienta auxiliar para auditorías puntuales.
- Su ejecución no es obligatoria en el pipeline, ya que la validación principal (validacion_dataset_modelado.py) garantiza la integridad global.
- Se recomienda utilizarlo si se desea revisar en detalle la trazabilidad de los outliers o documentar auditorías específicas.


📊 **Resultados de la comprobación auxiliar de outliers y clústeres**
Se ejecutó el script `check_outliers_clusters.py` para verificar la coherencia de los *outliers* en relación con los clústeres.  

**Principales hallazgos:**
- ✔️ **Distribución de clústeres**: se identificaron 4 valores (0–3), con asignación equilibrada y sin anomalías.
- ✔️ **No-outliers**: las columnas `cluster` y `__cluster__` son idénticas para todos los productos → confirmada la coherencia.
- ✔️ **Outliers**: 201 productos fueron marcados como outliers, y todos ellos fueron asignados de forma determinista al clúster mayoritario (`__cluster__ = 1`).

**Implicaciones:**
- La asignación determinista a clúster 1 asegura que ningún producto queda fuera del pipeline de modelado.
- La validación confirma que no existen inconsistencias entre `cluster` y `__cluster__` en los productos no-outlier.
- La estrategia adoptada (incluir outliers como parte del clúster mayoritario) se mantiene válida y no compromete la coherencia metodológica.

### **7.2. Preparación de los datos para el modelado.**



Tras validar la integridad del dataset en el apartado 7.1, el siguiente paso consiste en **normalizar y depurar la estructura de datos** para que pueda ser utilizada directamente en el entrenamiento de los modelos.

🎯 **Objetivo**.
- Unificar nombres de columnas clave.
- Eliminar duplicados y redundancias.
- Definir explícitamente el target y las features.
- Generar un dataset limpio y homogéneo que sirva como input estándar para todos los modelos.

🔁 **Pasos realizados**.
1. **Renombrado de columnas:**
   - `__cluster__` → `cluster_id`  
   - `demand_final_noised` → `sales_quantity`  

2. **Eliminación de duplicados:**
   - Se descartan `cluster` y `__product_id__`, ya que eran copias redundantes de `__cluster__` y `product_id`.

3. **Selección de variables explicativas (features):**
   - Precio: `precio_medio`, `price_virtual`, `price_factor_effective`, `demand_day_priceadj`.  
   - Factores externos: `m_agosto_nonprice`, `m_competition`, `m_inflation`, `m_promo`, entre otros.  
   - Outliers y trazabilidad: `is_outlier`, `tipo_outlier_year`, `decision_outlier_year`.  
   - Identificadores y fecha: `product_id`, `cluster_id`, `date`.

4. **Control de consistencia:**
   - Verificación de ausencia de duplicados en (`product_id`, `date`).  
   - Confirmación de que no existen valores nulos en la variable objetivo (`sales_quantity`).

5. **Exportación:**
   - Se genera el dataset final `data/processed/dataset_modelado_ready.parquet`, que será utilizado de manera uniforme en todos los experimentos de modelado.

🧪 **Resultado**.
El dataset preparado garantiza una **base coherente, sin ambigüedades ni redundancias**, y con una estructura estable que facilita:
- La aplicación consistente de modelos estadísticos y de machine learning.  
- La reproducibilidad de los experimentos (todos los modelos parten de la misma entrada).  
- La trazabilidad de resultados (columnas de target y features claramente identificadas).


⚙️ **Script: `preparacion_dataset_modelado.py`**

🎯 **Objetivo.**  
Normalizar y depurar el dataset de partida para que quede listo para el modelado, eliminando redundancias y asegurando que la estructura sea homogénea y estable.

➡️ **Entradas.**
- `data/processed/subset_modelado.parquet`

⬅️ **Salidas.**
- `data/processed/dataset_modelado_ready.parquet` (dataset final listo para modelado).

🔁 **Flujo de trabajo.**
1. **Renombrado de columnas clave**  
   - `__cluster__` → `cluster_id`  
   - `demand_final_noised` → `sales_quantity`  

2. **Eliminación de columnas redundantes**  
   - `cluster` (duplicado de `__cluster__`),  
   - `__product_id__` (duplicado de `product_id`),  
   - `demand_final_noiseds_adj` (columna auxiliar no utilizada).  

3. **Normalización de tipos**  
   - `date` → formato datetime.  
   - `product_id` → string.  
   - `cluster_id` → entero (`int` o `Int64` si hay nulos).  

4. **Control de duplicados y nulos**  
   - Eliminación de duplicados por (`product_id`, `date`).  
   - Filtrado de posibles nulos en `sales_quantity`.  

5. **Selección de variables finales**  
   - Identificadores y target: `product_id`, `date`, `cluster_id`, `sales_quantity`.  
   - Features de precio, factores externos y trazabilidad (`precio_medio`, `price_virtual`, `m_promo`, `is_outlier`, etc.).  
   - Ordenación por (`product_id`, `date`).  

6. **Exportación**  
   - Se guarda el dataset consolidado en `data/processed/dataset_modelado_ready.parquet`.  

📝 **Notas.**
- Este dataset es la **base de referencia para todos los modelos** de la Fase 7, evitando revalidaciones y asegurando consistencia.  
- La eliminación de redundancias y la normalización de tipos garantizan la trazabilidad y reproducibilidad de los resultados.  


In [10]:

# =============================================================================
# Script: preparacion_dataset_modelado.py
# =============================================================================

from __future__ import annotations
from pathlib import Path
import argparse
import logging
import pandas as pd

# ---------- Helper: localizar raíz del repo (busca data/processed hacia arriba)
def find_repo_root(start: Path | None = None) -> Path:
    p = Path(start or Path.cwd()).resolve()
    for parent in (p, *p.parents):
        if (parent / "data" / "processed").exists():
            return parent
    return p  # fallback

# ---------- Entorno (sirve para script y notebook)
if "__file__" in globals():
    _start = Path(__file__).resolve().parent
    LOGGER_NAME = Path(__file__).stem
else:
    _start = Path.cwd()
    LOGGER_NAME = "notebook.preparacion_dataset_modelado"

ROOT_DIR = find_repo_root(_start)
PROCESSED_DIR = ROOT_DIR / "data" / "processed"

# ---------- Logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(name)s | %(message)s",
)
log = logging.getLogger(LOGGER_NAME)

# ---------- Config “suave”: columnas a eliminar/renombrar/usar si existen
RENAME_MAP = {
    "__cluster__": "cluster_id",
    "demand_final_noised": "sales_quantity",
}
DROP_CANDIDATES = [
    "cluster",               # duplicado: nos quedamos con __cluster__ -> cluster_id
    "__product_id__",        # duplicado de product_id
    "demand_final_noiseds_adj",  # columna auxiliar que no aporta
]
# Features recomendadas (se usará la intersección para evitar KeyError)
FEATURES_RECOMENDADAS = [
    # ids & fecha (estos los forzamos aparte)
    # target -> sales_quantity (tras renombrado)
    "precio_medio",
    "price_virtual",
    "price_factor_effective",
    "demand_day_priceadj",
    # factores externos
    "m_agosto_nonprice",
    "m_competition",
    "m_inflation",
    "m_promo",
    # trazabilidad/outliers (opcionales, según uso como exógenas)
    "is_outlier",
    "tipo_outlier_year",
    "decision_outlier_year",
]

# ---------- Núcleo ------------------------------------------------------------
def prepare_dataset(df: pd.DataFrame) -> pd.DataFrame:
    """Aplica la preparación para modelado y devuelve el DataFrame listo."""
    df = df.copy()

    # 1) Renombrados (solo si existen)
    cols_a_renombrar = {c: n for c, n in RENAME_MAP.items() if c in df.columns}
    df = df.rename(columns=cols_a_renombrar)

    # Validaciones mínimas
    required = {"product_id", "date", "cluster_id", "sales_quantity"}
    missing = [c for c in required if c not in df.columns]
    if missing:
        raise ValueError(f"Faltan columnas requeridas tras renombrado: {missing}")

    # 2) Eliminar columnas redundantes si existen
    to_drop = [c for c in DROP_CANDIDATES if c in df.columns]
    if to_drop:
        log.info("Eliminando columnas redundantes: %s", to_drop)
        df = df.drop(columns=to_drop)

    # 3) Normalizar tipos
    df["date"] = pd.to_datetime(df["date"])
    df["product_id"] = df["product_id"].astype(str)
    # cluster_id como int (permitiendo nulos si los hubiera por seguridad)
    if df["cluster_id"].isna().any():
        df["cluster_id"] = df["cluster_id"].astype("Int64")
    else:
        df["cluster_id"] = df["cluster_id"].astype(int)

    # 4) Control de duplicados por (product_id, date)
    dups = df.duplicated(["product_id", "date"])
    n_dup = int(dups.sum())
    if n_dup > 0:
        log.warning("Se detectaron %s duplicados (product_id, date). Se conservará el primero.", n_dup)
        df = df.loc[~dups].copy()

    # 5) Verificación de nulos en target
    n_null_target = int(df["sales_quantity"].isna().sum())
    if n_null_target > 0:
        log.warning("Se encontraron %s nulos en sales_quantity. Filtrando filas nulas.", n_null_target)
        df = df.loc[df["sales_quantity"].notna()].copy()

    # 6) Selección de columnas finales (intersección segura)
    keep_base = ["product_id", "date", "cluster_id", "sales_quantity"]
    keep_feats = [c for c in FEATURES_RECOMENDADAS if c in df.columns]
    cols_finales = keep_base + keep_feats
    df = df[cols_finales].sort_values(["product_id", "date"]).reset_index(drop=True)

    return df

# ---------- CLI ---------------------------------------------------------------
def _parse_args() -> argparse.Namespace:
    p = argparse.ArgumentParser(description="Preparación del dataset para modelado.")
    p.add_argument("--in",  dest="inp",  type=str, default=str(PROCESSED_DIR / "subset_modelado.parquet"),
                   help="Ruta de entrada (PARQUET).")
    p.add_argument("--out", dest="outp", type=str, default=str(PROCESSED_DIR / "dataset_modelado_ready.parquet"),
                   help="Ruta de salida (PARQUET).")
    # Ignora flags de Jupyter si corre dentro de notebook
    args, _ = p.parse_known_args()
    return args

def run_prep(inp: str | Path = None, outp: str | Path = None) -> str:
    """Atajo para usar desde notebook o como función."""
    inp_path = Path(inp) if inp else (PROCESSED_DIR / "subset_modelado.parquet")
    out_path = Path(outp) if outp else (PROCESSED_DIR / "dataset_modelado_ready.parquet")

    log.info("Leyendo: %s", inp_path)
    df = pd.read_parquet(inp_path)

    log.info("Preparando dataset…")
    df_ready = prepare_dataset(df)

    out_path.parent.mkdir(parents=True, exist_ok=True)
    df_ready.to_parquet(out_path, index=False)
    log.info("Guardado dataset listo para modelado en: %s", out_path)

    return str(out_path)

def main() -> None:
    args = _parse_args()
    run_prep(args.inp, args.outp)

if __name__ == "__main__":
    main()


2025-09-08 12:29:42,828 | INFO | notebook.preparacion_dataset_modelado | Leyendo: C:\Users\crisr\Desktop\Máster Data Science & IA\PROYECTO\PFM2_Asistente_Compras_Inteligente\data\processed\subset_modelado.parquet
2025-09-08 12:29:43,464 | INFO | notebook.preparacion_dataset_modelado | Preparando dataset…
2025-09-08 12:29:43,837 | INFO | notebook.preparacion_dataset_modelado | Eliminando columnas redundantes: ['cluster', '__product_id__', 'demand_final_noiseds_adj']
2025-09-08 12:29:46,970 | INFO | notebook.preparacion_dataset_modelado | Guardado dataset listo para modelado en: C:\Users\crisr\Desktop\Máster Data Science & IA\PROYECTO\PFM2_Asistente_Compras_Inteligente\data\processed\dataset_modelado_ready.parquet


✅ **Verificación post-transformación del dataset listo para modelado.**

Tras la preparación del dataset (`dataset_modelado_ready.parquet`), se realiza una verificación ligera para asegurar que la transformación **no ha introducido errores** y que la estructura final es apta para los modelos.

**Qué se comprueba:**
- **Cobertura temporal:** las fechas abarcan el rango esperado (2022-01-01 → 2024-12-31).
- **Target (`sales_quantity`):** sin valores **nulos** ni **negativos**.  
  > No se validan los **ceros** porque son coherentes con días sin ventas.
- **Identificador (`product_id`):** sin nulos, sin valores “0” ni cadenas vacías.
- **Duplicados:** no existen duplicados en la combinación (`product_id`, `date`).
- **Clúster (`cluster_id`):** sin valores nulos y con valores dentro del rango esperado.

**Por qué es necesaria esta verificación:**
- Cada transformación (renombrados, drops, normalización) puede introducir errores de forma accidental.
- Esta comprobación actúa como **“post-check”** del bloque 7.2 y da garantías de que el dataset preparado mantiene la **integridad y consistencia** exigidas por el pipeline de modelado.

> Esta verificación es **operativa** y se mantiene en el **notebook** (no forma parte del pipeline en scripts) para agilizar el trabajo exploratorio y la defensa del proyecto.

In [11]:
# =============================================================================
# Script: preparacion_dataset_modelado.py
# =============================================================================

import pandas as pd
from pathlib import Path

# === Config ===
PATH_READY = Path(r"C:\Users\crisr\Desktop\Máster Data Science & IA\PROYECTO\PFM2_Asistente_Compras_Inteligente\data\processed\dataset_modelado_ready.parquet")
FECHA_MIN_ESPERADA = pd.Timestamp("2022-01-01")
FECHA_MAX_ESPERADA = pd.Timestamp("2024-12-31")

# === Carga ===
df = pd.read_parquet(PATH_READY)

# Asegurar tipos
df["date"] = pd.to_datetime(df["date"])
df["product_id"] = df["product_id"].astype(str)

print("=== Cobertura temporal ===")
print("Fecha mínima:", df["date"].min())
print("Fecha máxima:", df["date"].max())
cobertura_ok = (df["date"].min() <= FECHA_MIN_ESPERADA) and (df["date"].max() >= FECHA_MAX_ESPERADA)
print("Cobertura dentro del rango esperado:", cobertura_ok)

print("\n=== sales_quantity (target) ===")
print("Nulos:", int(df["sales_quantity"].isna().sum()))
print("Negativos:", int((df["sales_quantity"] < 0).sum()))
target_ok = (df["sales_quantity"].isna().sum() == 0) and ((df["sales_quantity"] < 0).sum() == 0)
print("Target OK (sin nulos ni negativos):", target_ok)

print("\n=== product_id ===")
print("Nulos:", int(df["product_id"].isna().sum()))
print("Valores '0':", int((df["product_id"] == "0").sum()))
print("Vacíos (''):", int((df["product_id"].str.len() == 0).sum()))
print("Únicos:", df["product_id"].nunique())
pid_ok = (df["product_id"].isna().sum() == 0) and ((df["product_id"] == "0").sum() == 0) and ((df["product_id"].str.len() == 0).sum() == 0)
print("product_id OK (no nulos/0/vacíos):", pid_ok)

print("\n=== Duplicados (product_id, date) ===")
dup_count = int(df.duplicated(["product_id", "date"]).sum())
print("Duplicados:", dup_count)
dups_ok = dup_count == 0
print("Sin duplicados pid+date:", dups_ok)

print("\n=== cluster_id ===")
print("Nulos:", int(df["cluster_id"].isna().sum()))
vals = sorted(pd.Series(df["cluster_id"].dropna().unique()).tolist())
print("Valores únicos:", vals)
cluster_ok = df["cluster_id"].isna().sum() == 0
print("cluster_id OK (sin nulos):", cluster_ok)

print("\n=== Resumen (OK=True) ===")
checks = {
    "cobertura_temporal_ok": cobertura_ok,
    "target_ok": target_ok,
    "product_id_ok": pid_ok,
    "sin_duplicados_pid_date": dups_ok,
    "cluster_ok": cluster_ok,
}
for k, v in checks.items():
    print(f"{k}: {bool(v)}")

=== Cobertura temporal ===
Fecha mínima: 2022-01-01 00:00:00
Fecha máxima: 2024-12-31 00:00:00
Cobertura dentro del rango esperado: True

=== sales_quantity (target) ===
Nulos: 0
Negativos: 0
Target OK (sin nulos ni negativos): True

=== product_id ===
Nulos: 0
Valores '0': 0
Vacíos (''): 0
Únicos: 3596
product_id OK (no nulos/0/vacíos): True

=== Duplicados (product_id, date) ===
Duplicados: 0
Sin duplicados pid+date: True

=== cluster_id ===
Nulos: 0
Valores únicos: [0, 1, 2, 3]
cluster_id OK (sin nulos): True

=== Resumen (OK=True) ===
cobertura_temporal_ok: True
target_ok: True
product_id_ok: True
sin_duplicados_pid_date: True
cluster_ok: True


📊 **Resultados de la verificación post-transformación**.

La verificación realizada sobre el dataset `dataset_modelado_ready.parquet` confirma que la transformación no introdujo errores y que la estructura final es **coherente y apta para el modelado**.

**Hallazgos principales:**
- ✔️ **Cobertura temporal completa:** fechas desde 2022-01-01 hasta 2024-12-31.  
- ✔️ **Target (`sales_quantity`):** sin nulos ni valores negativos. Los ceros se mantienen como representación válida de días sin ventas.  
- ✔️ **Product_ID:** sin nulos, sin valores inválidos (0 o cadenas vacías). Se identifican 3.596 productos únicos.  
- ✔️ **Duplicados:** no existen duplicados en la combinación (`product_id`, `date`).  
- ✔️ **Cluster_ID:** todos los productos tienen clúster asignado (0–3), sin nulos ni valores fuera de rango.  

> **Conclusión:**  
El dataset preparado conserva la integridad y consistencia requeridas.  
Esto asegura que el archivo `dataset_modelado_ready.parquet` puede utilizarse como **input único y estable** en todos los experimentos de la Fase 7, garantizando trazabilidad, reproducibilidad y ausencia de sesgos estructurales.